In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import sklearn as skl
import sys

In [4]:
np.set_printoptions(threshold=sys.maxsize)

In [5]:
path = "./data/"

In [6]:
filename_in = filename = "./a.txt"

In [7]:
def file_reader(path,filename):
    with open(path+filename, "r") as f:
        lines = f.readlines()
    D, I, S, V, F = list(map(int, lines[0].strip().split(" ")))
    return D, I, S, V, F, lines

# Street Reader

In [8]:
def street_reader(D, I, S, V, F, lines):
#     with open(path+filename, "r") as f:
#         lines = f.readlines()
#     D, I, S, V, F = list(map(int, lines[0].strip().split(" ")))
    streets = pd.DataFrame()
    raw_data = []
    for s in tqdm(range(1,S+1)):
        data = {}
        elems = list(map(str, lines[s].strip().split(" ")))
        data['B'] = int(elems[0])
        data['E'] = int(elems[1])
        data['street_name'] = elems[2]
        data['L'] = int(elems[3])
        raw_data.append(data)
    
    streets = streets.append(raw_data, ignore_index=True)
    streets = streets.astype({'B':int,'E':int,'L':int})
    return streets

# Read cars data

In [9]:
def car_reader(D, I, S, V, F, lines):
    cars = pd.DataFrame()
    raw_data = []
    for v in range(S+1,V+S+1):
        data = {}
        elems = list(map(str, lines[v].strip().split(" ")))
        data['P'] = int(elems[0])
#         for p in range(data['P']):
#             data['street_'+str(p+1)] = elems[p+1]
        data['all_streets'] = elems[1:]
        raw_data.append(data)     
    cars = cars.append(raw_data, ignore_index=True)
    return cars

In [168]:
D, I, S, V, F = file_reader(path,filename)
cars = car_reader(D, I, S, V, F)

In [169]:
cars

,P,all_streets
0,4,"[rue-de-londres, rue-d-amsterdam, rue-de-mosco..."
1,3,"[rue-d-athenes, rue-de-moscou, rue-de-londres]"


# Fuck it

## Método 1: Solo una calle por nodo

In [220]:
def only_the_one(filename):
    '''
    Método 1: Nos quedamos solo con 1 calle random 
    por cada nodo fin (E) y la ponemos en verde.
    '''
    D, I, S, V, F, lines = file_reader(path,filename)
    streets = street_reader(D, I, S, V, F, lines)
    
    streets_solo_1 = streets.drop_duplicates(subset=['E'])
    
    is_n = streets_solo_1['E'].value_counts()
    
    with open(path_out+filename+'.out', "w") as the_file:
        the_file.write(f"{len(streets_solo_1)} \n")
        for n in streets_solo_1['E'].unique():
            the_file.write(f"{n} \n")
            the_file.write(f"{is_n[n]} \n")
            for s_e in streets_solo_1[streets_solo_1['E']==n].to_numpy():
                the_file.write(f"{s_e[2]+' '+str(D)} \n")

In [221]:
## Ejecutar el método
path_out = path+'out/metodo_1/'
for FILE in os.listdir(path):
    if not FILE.endswith(".txt"):
        continue
        
    print(f"PROCESANDO FICHERO -> {FILE}")
    
    only_the_one(FILE)

100%|██████████████████████████████████| 9102/9102 [00:00<00:00, 379224.74it/s]

PROCESANDO FICHERO -> a.txt
PROCESANDO FICHERO -> b.txt



100%|████████████████████████████████| 35030/35030 [00:00<00:00, 515296.42it/s]

PROCESANDO FICHERO -> c.txt



 53%|█████████████████               | 51317/95928 [00:00<00:00, 508092.35it/s]

PROCESANDO FICHERO -> d.txt


100%|████████████████████████████████████| 998/998 [00:00<00:00, 332928.93it/s]

PROCESANDO FICHERO -> e.txt



100%|████████████████████████████████| 10000/10000 [00:00<00:00, 500680.90it/s]

PROCESANDO FICHERO -> f.txt


## Método 2: ¿?

In [224]:
D, I, S, V, F, lines = file_reader(path,'b.txt')

In [225]:
streets = street_reader(D, I, S, V, F, lines)

100%|██████████████████████████████████| 9102/9102 [00:00<00:00, 350034.89it/s]


In [226]:
street_dict = {i:0 for i in streets['street_name'].unique()}

In [228]:
street_dict['bjfg-dic']

0

In [179]:
D, I, S, V, F = file_reader(path,filename)
cars = car_reader(D, I, S, V, F)

In [184]:
for car_st in cars['all_streets']:
    for st in car_st:
        street_dict[st] += 1

In [195]:
street_dict['rue-d-amsterdam']=4

In [200]:
street_dict

{'rue-de-londres': 2,
 'rue-d-amsterdam': 4,
 'rue-d-athenes': 1,
 'rue-de-rome': 1,
 'rue-de-moscou': 2}

In [201]:
streets['street_count'] = streets['street_name'].map(street_dict)

In [202]:
streets = streets.sort_values(['street_count'],ascending=False)

In [203]:
streets_best = streets.drop_duplicates(subset=['E'])

In [204]:
streets_best

,B,E,street_name,L,street_count
1,0,1,rue-d-amsterdam,1,4
0,2,0,rue-de-londres,1,2
4,1,2,rue-de-moscou,3,2
3,2,3,rue-de-rome,2,1


In [239]:
def m_2(filename):
    D, I, S, V, F, lines = file_reader(path,filename)
    streets = street_reader(D, I, S, V, F, lines)
    street_dict = {i:0 for i in streets['street_name'].unique()}
    cars = car_reader(D, I, S, V, F, lines)
    for car_st in tqdm(cars['all_streets']):
        for st in car_st:
            street_dict[st] += 1
        streets['street_count'] = streets['street_name'].map(street_dict)
        streets = streets.sort_values(['street_count'],ascending=False)
        streets_best = streets.drop_duplicates(subset=['E'])
    
    is_n = streets_best['E'].value_counts()
    
    with open(path_out+filename+'.out', "w") as the_file:
        the_file.write(f"{len(streets_best)} \n")
        for n in tqdm(streets_best['E'].unique()):
            the_file.write(f"{n} \n")
            the_file.write(f"{is_n[n]} \n")
            for s_e in streets_best[streets_best['E']==n].to_numpy():
                the_file.write(f"{s_e[2]+' '+str(D)} \n")

In [237]:
m_2('b.txt')

100%|████████████████████████████████████| 7073/7073 [00:03<00:00, 1813.13it/s]


In [240]:
## Ejecutar el método
path_out = path+'out/metodo_2/'
for FILE in os.listdir(path):
    if not FILE.endswith(".txt"):
        continue
        
    print(f"PROCESANDO FICHERO -> {FILE}")
    
    m_2(FILE)

  0%|                                                 | 0/1000 [00:00<?, ?it/s]

PROCESANDO FICHERO -> a.txt
PROCESANDO FICHERO -> b.txt


  0%|                                                 | 0/1000 [00:00<?, ?it/s]

PROCESANDO FICHERO -> c.txt


100%|████████████████████████████████| 95928/95928 [00:00<00:00, 518533.90it/s]

...


  4%|█▌                                     | 39/1000 [00:00<00:02, 382.35it/s]

PROCESANDO FICHERO -> e.txt


  0%|                                                 | 0/1000 [00:00<?, ?it/s]

PROCESANDO FICHERO -> f.txt


100%|████████████████████████████████████| 1662/1662 [00:00<00:00, 1812.44it/s]
